<a href="https://colab.research.google.com/github/souravs17031999/Projects-kaggle-problems-60daysofudacity/blob/master/Pysyft_federated_dataset_creation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# OBJECTIVE : To create custom federated datasets and dataloader in PySyft

In [0]:
!pip install syft

In [0]:
# import all required packages
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [0]:
# creating virtual workers
import syft as sy
hook = sy.TorchHook(torch)
# let's create two virtual workers who will hold the data while training the model locally
bob = sy.VirtualWorker(hook, id = "bob")
alice = sy.VirtualWorker(hook, id = "alice")
crypto_provider = sy.VirtualWorker(hook, id="crypto_provider")

W0825 04:16:33.146131 139690531305344 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/usr/local/lib/python3.6/dist-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0825 04:16:33.159694 139690531305344 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tf_encrypted/session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [0]:
# creating inputs and targets (labels)
inputs = torch.tensor([[0,0],[0,1],[1,0],[1,1.]], requires_grad=True)
target = torch.tensor([[0],[0],[1],[1.]], requires_grad=True)

In [0]:
# use BaseDataset class which takes data, targets as args
# This class has .data attribute , .targets attribute to make it able to manipulate
# It has .send attibute to send to workers
bob_train_dataset = sy.BaseDataset(inputs[:2], target[:2]).send(bob)
alice_train_dataset = sy.BaseDataset(inputs[2:], target[2:]).send(alice)

In [0]:
print(bob_train_dataset.data)
print(bob_train_dataset.targets)
print(alice_train_dataset.data)
print(alice_train_dataset.targets)

Tensor>[PointerTensor | me:55617970640 -> bob:12076290068]
Tensor>[PointerTensor | me:38801454586 -> bob:96929184740]
Tensor>[PointerTensor | me:93238096976 -> alice:11082570616]
Tensor>[PointerTensor | me:59498225150 -> alice:27819445055]


In [0]:
# FederatedDataset class takes a list of remote datasets as args
# It takes the datasets already sent to remote workers
# This also acts as input to FederatedDataLoader
federated_train_dataset = sy.FederatedDataset([bob_train_dataset, alice_train_dataset])

In [0]:
print(federated_train_dataset)

FederatedDataset
    Distributed accross: bob, alice
    Number of datapoints: 4



In [0]:
# This FederatedDataLoader class iterates over the objects created by FederatedDataset class
federated_train_loader = sy.FederatedDataLoader(federated_train_dataset, shuffle=True, batch_size=1)

In [0]:
inputs, labels = next(iter(federated_train_loader))

In [0]:
inputs.get()

tensor([[0., 0.]], requires_grad=True)

In [0]:
labels.get()

tensor([[0.]], requires_grad=True)